In [ ]:
from itertools import combinations

In [ ]:
def get_elements(solution):

    periods = [s for s in solution]
    matches = [m for s in periods for m in s]
    teams = [t for m in matches for t in m]

    return periods, matches, teams


def get_weeks(periods, n):
    return [[p[i] for p in periods] for i in range(n-1)]

In [ ]:
solution = ''

solution = eval(solution)
periods, solution_matches, teams = get_elements(solution)
errors = []

if len(errors) == 0 and len(solution) > 0:

    n = max(teams)

    teams_matches = combinations(set(teams),2)

    # every team plays with every other teams only once (print the duplicate matches found) 
    duplicates = [match for h,a in teams_matches if solution_matches.count([h,a]) + solution_matches.count([a,h]) > 1 for match in [[h,a], [a,h]] if solution_matches.count(match) > 0]
    if duplicates:
        errors.append('There are duplicated matches')
        print(f'Duplicates found: {duplicates}')

    # each team cannot play against itself
    if any([h==a for h,a in solution_matches]):
        errors.append('There are self-playing teams')

    weeks = get_weeks(periods, n)

    # every team plays once a week
    teams_per_week = [[j for i in w for j in i] for w in weeks]
    if any([len(tw) != len(set(tw)) for tw in teams_per_week]):
        errors.append('Some teams play multiple times in a week')

    teams_per_period = [[j for i in p for j in i] for p in periods]

    # every team plays at most twice during the period
    if any([any([tp.count(elem) > 2 for elem in tp]) for tp in teams_per_period]):
        errors.append('Some teams play more than twice in the period')

if len(errors) == 0:
    print('Valid solution')
else: 
    print(errors) 

Valid solution


In [10]:
import os
import json
import sys

def load_json(path):
    try:
        with open(path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error reading {path}: {e}")
        sys.exit(1)

def get_table_value(data_dict, key, metric):
    """
    Extracts value based on metric ('time' or 'obj') and applies cleaning rules.
    """
    if key not in data_dict:
        return "-" # Key missing in this file/instance
    
    entry = data_dict[key]
    
    if metric == 'time':
        val = entry.get('time')
        # Rule: if time is 300, set to NA
        if val == 300:
            return "NA"
        return str(val)
        
    elif metric == 'obj':
        val = entry.get('obj')
        # Rule: if obj is 'null' (string) or None, set to NA
        if val == "null" or val is None:
            return "NA"
        return str(val)
    
    return str(entry.get(metric, ""))

def write_markdown_table(file_handle, title, headers, rows):
    """
    Writes a formatted markdown table to the provided file handle.
    """
    file_handle.write(f"### {title}\n")
    
    # Header Row
    header_str = "| N | " + " | ".join(headers) + " |"
    file_handle.write(header_str + "\n")
    
    # Separator Row
    sep_str = "|---| " + " | ".join(["---"] * len(headers)) + " |"
    file_handle.write(sep_str + "\n")
    
    # Data Rows
    for n_val, row_data in rows:
        row_str = f"| {n_val} | " + " | ".join(row_data) + " |"
        file_handle.write(row_str + "\n")
    
    file_handle.write("\n")

def main():
    directory = '/home/hsn/projects/sports_tournament_scheduling/res/CP'
    output_filename = 'summary_tables.md'
    output_path = os.path.join(directory, output_filename)
    
    print(f"Scanning directory: {directory}")
    
    # 1. Collect all data and sort by instance size (N)
    files = [f for f in os.listdir(directory) if f.endswith('.json')]
    
    # Sort files numerically based on the integer before .json (e.g. 6.json before 10.json)
    try:
        files.sort(key=lambda x: int(x.split('.')[0]))
    except ValueError:
        files.sort()

    all_data = [] # List of tuples (N, json_data)
    all_keys = set()

    for f in files:
        path = os.path.join(directory, f)
        data = load_json(path)
        
        # Extract N from filename (e.g., "6.json" -> 6)
        n_val = f.split('.')[0] 
        
        all_data.append((n_val, data))
        all_keys.update(data.keys())

    sorted_all_keys = sorted(list(all_keys))

    # 2. Classify Keys into the 4 Categories
    is_base = lambda k: 'reg' in k or 'sb' in k
    
    # Table 1: Base only (No 'ss', No 'opt')
    keys_t1 = [k for k in sorted_all_keys if is_base(k) and 'ss' not in k and 'opt' not in k]
    
    # Table 2: Opt only (Has 'opt', No 'ss')
    keys_t2 = [k for k in sorted_all_keys if is_base(k) and 'ss' not in k and 'opt' in k]
    
    # Table 3: SS only (Has 'ss', No 'opt')
    keys_t3 = [k for k in sorted_all_keys if is_base(k) and 'ss' in k and 'opt' not in k]
    
    # Table 4: SS and Opt (Has 'ss' and 'opt')
    keys_t4 = [k for k in sorted_all_keys if is_base(k) and 'ss' in k and 'opt' in k]

    # 3. Generate Data for each table
    rows_t1 = []
    for n, data in all_data:
        rows_t1.append((n, [get_table_value(data, k, 'time') for k in keys_t1]))

    rows_t2 = []
    for n, data in all_data:
        rows_t2.append((n, [get_table_value(data, k, 'obj') for k in keys_t2]))

    rows_t3 = []
    for n, data in all_data:
        rows_t3.append((n, [get_table_value(data, k, 'time') for k in keys_t3]))

    rows_t4 = []
    for n, data in all_data:
        rows_t4.append((n, [get_table_value(data, k, 'obj') for k in keys_t4]))

    # 4. Write to Markdown File
    try:
        with open(output_path, 'w') as f_out:
            f_out.write(f"# Results Summary\nGenerated from {len(files)} JSON files.\n\n")
            
            write_markdown_table(f_out, "Table 1: Time (Reg/SB only)", keys_t1, rows_t1)
            write_markdown_table(f_out, "Table 2: Objective (Opt + Reg/SB)", keys_t2, rows_t2)
            write_markdown_table(f_out, "Table 3: Time (SS + Reg/SB)", keys_t3, rows_t3)
            write_markdown_table(f_out, "Table 4: Objective (SS + Opt + Reg/SB)", keys_t4, rows_t4)
            
        print(f"Successfully saved tables to: {output_path}")
        
    except IOError as e:
        print(f"Error writing to file {output_path}: {e}")

if __name__ == "__main__":
    main()

Scanning directory: /home/hsn/projects/sports_tournament_scheduling/res/CP
Successfully saved tables to: /home/hsn/projects/sports_tournament_scheduling/res/CP/summary_tables.md


In [1]:
from pathlib import Path
import json
model_names = {
    "gecode_reg" :{ 'model':f'cp_model.mzn', 'solver':'gecode', 'opt':False  },
    "gecode_sb" :{ 'model':f'cp_model_sb.mzn', 'solver':'gecode', 'opt':False  },
    "gecode_opt_reg" :{ 'model':f'cp_optimal.mzn', 'solver':'gecode', 'opt':True  },
    # "gecode_opt_sb" :{ 'model':'cp_optimal_sb', 'solver':'gecode', 'opt':True  },
    # "chuffed_reg" :{ 'model':'cp_model', 'solver':'chuffed', 'opt':False  },
    # "chuffed_sb" :{ 'model':'cp_model_sb', 'solver':'chuffed', 'opt':False  },
    # "chuffed_opt_reg" :{ 'model':'cp_optimal', 'solver':'chuffed', 'opt':True  },
    # "chuffed_opt_sb" :{ 'model':'cp_optimal_sb', 'solver':'chuffed', 'opt':True  },
    # "cp_reg" :{ 'model':'cp_model', 'solver':'cp', 'opt':False  },
    # "cp_sb" :{ 'model':'cp_model_sb', 'solver':'cp', 'opt':False  },
    # "cp_opt_reg" :{ 'model':'cp_optimal', 'solver':'cp', 'opt':True  },
    # "cp_opt_sb" :{ 'model':'cp_optimal_sb', 'solver':'cp', 'opt':True  },
}

In [ ]:
# #!/usr/bin/env bash
# set -Eeuo pipefail
# trap cleanup SIGINT SIGTERM ERR EXIT

# # =============================
# # Configurable Parameters
# # =============================
# APPROACHES=("CP" "MIP" "SMT")
# # INSTANCES=(6 8 10 12 14 16)
# DEFAULT_INSTANCE=6

# # =============================
# # Utility Functions
# # =============================
# cleanup() {
#   trap - SIGINT SIGTERM ERR EXIT
# }

# msg() {
#   echo -e "[$(date '+%H:%M:%S')] ${1-}"
# }

# die() {
#   local msg=$1
#   local code=${2-1}
#   echo >&2 "Error: $msg"
#   exit "$code"
# }

# usage() {
#   cat << EOF
# Usage: $(basename "${BASH_SOURCE[0]}") [OPTIONS]

# Options:
#   -a, --approach    Approach to run (CP | MIP | SMT). Default: all available approaches
#   -n, --instance    Instance size (e.g., 6, 8, 10, 12). Leave empty for all 
#   -h, --help        Show this help and exit

# Examples:
#   ./entrypoint.sh --approach CP --instance 8
#   ./entrypoint.sh
# EOF
#   exit 0
# }

# # =============================
# # Parse Command-Line Arguments
# # =============================
# parse_params() {
#   SELECTED_APPROACH=""
#   INSTANCE=0

#   while [[ $# -gt 0 ]]; do
#     case "$1" in
#       -a|--approach)
#         SELECTED_APPROACH="$2"
#         shift 2
#         ;;
#       -n|--instance)
#         INSTANCE="$2"
#         shift 2
#         ;;
#       -h|--help)
#         usage
#         ;;
#       *)
#         die "Unknown option: $1"
#         ;;
#     esac
#   done
# }

# # =============================
# # Main Logic
# # =============================
# main() {
#   parse_params "$@"

# if [[ -z "$SELECTED_APPROACH" ]]; then
#   msg "(entrypoint) No specific approach selected — running all available approaches, with all instance sizes"
#   for ap in "${APPROACHES[@]}"; do
#     run_approach "$ap" "$inst"
#   done

#   # msg "(entrypoint) Now running solution checker on all generated solutions"
#   # for ap in "${APPROACHES[@]}"; do
#   #   local res="res/${ap}/"
#   #   python3 solution_checker.py "$res"
#   # done
#   #
# # elif [[ -z "$INSTANCE" ]]; then
# #   msg "(entrypoint) Approach '$SELECTED_APPROACH' selected — running with all instance sizes"
# #   for inst in ${INSTANCES[@]}; do
# #     run_approach "$SELECTED_APPROACH" "$inst"
# #   done
  
#   # msg "(entrypoint) Now running solution checker on all generated solutions"
#   # local res="res/${SELECTED_APPROACH}/"
#   # python3 solution_checker.py "$res"
#   #
# else
#   run_approach "$SELECTED_APPROACH" "$INSTANCE"
# fi

# msg "[entrypoint] All tasks completed successfully."
# }

# run_approach() {
#   local approach="$1"
#   local instance="$2"
#   local file="source/${approach}/run.py"
#   local res="res/${approach}/"

#   msg "(entrypoint) Running ${approach} approach with instance=${instance}"

#   if [[ -f "$file" ]]; then
#     python3 "$file" "-n" "$instance"
#     # msg "(entrypoint) Running solution checker on approach ${approach} with instance=${instance}"
#     # python3 solution_checker.py "$res"
#   else
#     msg "(entrypoint) File not found: $file — skipping ${approach} approach."
#   fi
# }

# # =============================
# # Entrypoint Execution
# # =============================
# main "$@"
